# Digit Recognizer (Kaggle), 0.991 Accuracy with Keras
<hr>
In this tutorial we are going to use __*Convolutional Neural Networks*__ to classify images from the __*MNIST*__ dataset.

- You can find the competition [here](https://www.kaggle.com/c/digit-recognizer/data)

In [2]:
# Load libraries
%pylab inline

import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
from sklearn.model_selection import train_test_split

import csv
from tqdm import tqdm
import numpy.random

Populating the interactive namespace from numpy and matplotlib


## Open the dataset
After downloading the dataset, we are going to do the following:

1. Open the file and load the data
2. Format the data and get the labels
3. Check for NaN values
4. Split the dataset to train and validation
5. Normalize the data

In [5]:
def open_train_data(path):
    
    train = [] 
    
    with open(path, 'r') as f:
        reader = csv.reader(f)
        lines = list(reader)
        for line in tqdm(lines[1:]):
            label = line[0]
            
            image = np.array([x for x in line[1:]])
            image = image.astype('float32')
            
            # Format the data to 28x28x1 (in grey scale)
            image = np.reshape(image, (28, 28, 1))
            train.append([image, label])
    
    return np.array(train)

In [6]:
def split_train_test(train):
    
    np.random.shuffle(train)
    
    features = [x[0] for x in train]
    labels = [x[1] for x in train]
    
    # Split the dataset to train and validation
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.025, random_state=42)
    
    # One-hot Encoding
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    
    return (np.array(x_train), y_train), (np.array(x_test), y_test)
    

In [7]:
# Load the data, run only once
# train = open_train_data('dataset/train.csv')
# np.save('train.npy', train)

In [8]:
# If you have already ran the the function open_train_data then run this
train = np.load('train.npy')

In [17]:
# Check for missing values
import pandas as pd

for idx, feature in enumerate(train):
    if pd.isnull(feature).any():
        print('Found NaN value in feature %d' % idx)
        break

In [18]:
(x_train, y_train), (x_test, y_test) = split_train_test(train)
x_train = x_train / 255.0
x_test = x_test / 255.0

In [19]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((40950, 28, 28, 1), (40950, 10), (1050, 28, 28, 1), (1050, 10))

In [20]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (2, 2), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Conv2D(128, (2, 2), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))
# model = Sequential()

# model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu', input_shape = (28,28,1)))
# model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.25))


# model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.25))


# model.add(Flatten())
# model.add(Dense(256, activation = "relu"))
# model.add(Dropout(0.5))
# model.add(Dense(10, activation = "softmax"))

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        4128      
_________________________________________________________________
activation_2 (Activation)    (None, 27, 27, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 64)        8256      
__________

In [37]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [38]:
# Compile the model
batch_size = 64

opt_rms = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# opt_rms = keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', 
              optimizer=opt_rms, 
              metrics=['accuracy'])

In [ ]:
from time import time
epochs = 50

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph/{}'.format(time()), histogram_freq=0, write_graph=True, write_images=True)
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', verbose=1, monitor='val_acc', save_best_only=True, mode='auto')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test), 
          callbacks=[tbCallBack, checkpoint])

Epoch 1/50
640/640 [==============================] - 20s 31ms/step - loss: 0.3599 - acc: 0.9129 - val_loss: 0.0890 - val_acc: 0.9686

Epoch 00001: val_acc improved from -inf to 0.96857, saving model to model-001.h5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.2426 - acc: 0.9408 - val_loss: 0.1124 - val_acc: 0.9676

Epoch 00002: val_acc did not improve from 0.96857
Epoch 3/50
640/640 [==============================] - 18s 29ms/step - loss: 0.2356 - acc: 0.9430 - val_loss: 0.1163 - val_acc: 0.9771

Epoch 00003: val_acc improved from 0.96857 to 0.97714, saving model to model-003.h5
Epoch 4/50
640/640 [==============================] - 19s 29ms/step - loss: 0.2301 - acc: 0.9462 - val_loss: 0.0833 - val_acc: 0.9762

Epoch 00004: val_acc did not improve from 0.97714
Epoch 5/50
640/640 [==============================] - 18s 29ms/step - loss: 0.2191 - acc: 0.9472 - val_loss: 0.0869 - val_acc: 0.9724

Epoch 00005: val_acc did not improve from 0.97714
Epoch 6/50

In [30]:
model.load_weights('model-035.h5')

In [31]:
# Load the test data
def open_test_data(path):
    
    test = [] 
    
    with open(path, 'r') as f:
        reader = csv.reader(f)
        lines = list(reader)
        image_number = 1
        for line in tqdm(lines[1:]):
            
            image = np.array([x for x in line])
            image = image.astype('float32')
            image = np.reshape(image, (28, 28, 1))
            test.append([image, image_number])
            image_number += 1
    
    return np.array(test)

In [27]:
# test_data = open_test_data('dataset/test.csv')

100%|██████████| 28000/28000 [00:10<00:00, 2552.49it/s]


In [28]:
# np.save('test.npy', test_data)

In [32]:
test_data = np.load('test.npy')

In [33]:
import matplotlib.pyplot as plt

with open('submission.csv', 'w') as f:
    f.write('ImageId,Label\n')
    for data in tqdm(test_data):
        arr = numpy.expand_dims(data[0], axis=0)
        number = model.predict(arr)
        
        label = argmax(number)
        f.write(str(data[1]) + ',' + str(label) + '\n')

100%|██████████| 28000/28000 [00:52<00:00, 534.47it/s]
